In [1]:
#!pip install pyyaml

In [2]:
import json
import requests
import yaml
import time

In [3]:
# Load YAML file
def load_config(filename):
    with open(filename, 'r') as file:
        config = yaml.safe_load(file) 
    return config

config = load_config('config.yaml')

In [4]:
API_KEY = config['API_KEY']
API_VERSION = config['API_VERSION']
RESOURCE_ENDPOINT = config['RESOURCE_ENDPOINT']

In [5]:
# These two functions are helper functions
def post_request(url, headers, body):
    response = requests.post(url, headers=headers, data=body)
        
    response.raise_for_status()
    return response

def exception_code(retries, deployment_id, e):
    MAX_RETRIES = 3
    RETRY_SECS = 1
    if retries >= MAX_RETRIES:
        print(f'Failed attempt {retries+1} of {MAX_RETRIES+1}.')
        print(error_msg)
        raise Exception(f"Test failed for deployment: {deployment_id}, Error received: {e}")
    else:
        print(f'Failed attempt {retries+1} of {MAX_RETRIES + 1}. Waiting {RETRY_SECS} secs before next attempt...')
        
    retries += 1
    time.sleep(RETRY_SECS)
    
    return retries

In [6]:
def text_completion(prompt, simulate_api_error=False):
  
    completions_url = f"{RESOURCE_ENDPOINT}/openai/deployments/gpt-35-turbo-instruct/completions?api-version={API_VERSION}"
    body = json.dumps({
        "prompt": prompt,
         "max_tokens": 250,  # Limit the response
    })
    headers = {
        'Content-Type': 'application/json',
        'api-key': API_KEY
    }

    retries = 0

    while True:
        try:
            response = post_request(completions_url, headers, body)

            #print('User: ', prompt)
            print(json.loads(response.text).get('choices')[0].get('text'))
            break
        except Exception as e:
            retries = exception_code(retries, 'gpt-35-turbo-instruct', e)

In [7]:
text = """
You were a blaze and then the moment went You tried to solve filthy patches But now you're gonna win Oh man, oh he was here Oh oh oh oh oh 
man He got the two girls in your house Yup, you smell like smoke I know, I'm sorry, I'm trying to quit Oh man! Here, try these bolt high 
quality electronic cigarettes From smokeless image, they use a battery and E-liquid So there's no tobacco, no tar, no ash So there are no
stinky smells Thank you, smokeless, I'm ex Now I'm so bad for man Be sure to tune in next time For a hot man, a healer's hero A hot man, he 
got the two girls in your house A hot man, a healer's hero
"""


In [8]:
classification_query = "Pick the 5 most likely classifications for this document, and return as a csv format, with the classification first and the numerical probability second: "

In [9]:
text_completion(classification_query + " " + text)


No way, 0.9
Teens, 0.7
Anti-Smoking, 0.7
Peer Pressure, 0.6
Youth Culture, 0.5


In [10]:
sentiment_query = "Assess the sentiment as a floating point value from -1 to 1 and magnitude as a floating point from 0 to 1, format as (sentiment, magnitude) with no other text in response:"

In [11]:
text_completion(sentiment_query + " " + text)


(0.8, 0.8)


In [12]:
summarization_query = "Summarize the document using complete sentences in less than 200 words, covering main points, to be used for quick scan by researchers: "

In [13]:
text_completion(summarization_query + " " + text)


A group of teenagers discuss the idea that smoking makes you cool. They reject this belief and explain their reasons for not smoking. They believe that being true to themselves is more important than trying to fit in by smoking. They share their personal experiences with peer pressure and how they have chosen to not give in to it. One teenager in the group admits to trying cigarettes when they were 14, but stopped because they didn't enjoy it and didn't feel the need to fit in. They all agree that smoking is not necessary to be different or cool.


In [23]:
from openai import AzureOpenAI
deployment = 'gpt-4o-2024-05-13'

client = AzureOpenAI(
    api_key=API_KEY,
    api_version=API_VERSION,
    azure_endpoint=RESOURCE_ENDPOINT,
)
image_url = "https://github.com/geoffswc/Libguide-AI-Tools-Archival-Research/raw/main/cartoon_frame_jpg.jpg"
messages = [
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "what objects do you detect in this image?"},
        {
          "type": "image_url",
          "image_url": {
            "url": image_url,
          },
        },
      ],
    }
  ]

response = client.chat.completions.create(
  model=deployment,
  messages=messages,
  max_tokens=300,
)

In [24]:
print(response.choices[0].message.content)

In the image, I detect a superhero character, electronic cigarettes, city buildings, a logo and text for Smokeless Image Electronic Cigarettes, a speech bubble, and an arm holding an electronic cigarette. Additionally, there are comic book-style graphics, a character labeled "Analog" with a broken wall in the background, and a "To be continued" banner.
